In [1]:
import sys
import gc

import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
import os
os.environ["TOKENIZERS_PARALLELISM"] = "False"

In [2]:
import sklearn
import lightgbm

print(sklearn.__version__)
print(lightgbm.__version__)

1.2.2
3.3.2


In [3]:
test = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/test_essays.csv')
sub = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/sample_submission.csv')
org_train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/train_essays.csv')

In [4]:
train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/daigt/train_v2_drcat_02.csv', sep=',')
display(train.shape);

(44868, 5)

#### Read and append the generated data

In [5]:
train_gen_data = pd.read_parquet('/home/peng_sun2/s3shared/kaggle/llm-2023/external_data/gen_data_21122023.parquet')
display(train_gen_data.shape)

(18894, 5)

In [6]:
train = train.drop_duplicates(subset = ['text'])
train.reset_index(drop = True, inplace = True)

### append the generated data
# train = pd.concat([train, train_gen_data], axis = 0).reset_index(drop = True)
# display(train.shape)

In [7]:
LOWERCASE = False
VOCAB_SIZE = 30522

In [8]:
## Create byte-pair encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token='[UNK]'))

In [9]:
## adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()]+ [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

In [10]:
# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size = VOCAB_SIZE, special_tokens = special_tokens)

In [11]:
# creating huggingface dataset object
dataset = Dataset.from_pandas(test[['text']])
def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i:i+1000]['text']
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer = trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []
for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pyarrow/pandas_compat.py:358: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/44868 [00:00<?, ?it/s]

In [12]:
tokenized_texts_test[1]

['ĠBbb', 'Ġccc', 'Ġddd', '.']

In [13]:
def dummy(text):
    return text

In [14]:
vectorizer = TfidfVectorizer(ngram_range = (3, 5), lowercase=False, sublinear_tf = True, analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None,
                            strip_accents = 'unicode')
vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, 
                             #vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode'
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)
## get the vocabulary of the training dataset
train_vocab = vectorizer.vocabulary_

del vectorizer
gc.collect()

{'ĠAaa Ġbbb Ġccc': 0, 'Ġbbb Ġccc .': 6, 'ĠAaa Ġbbb Ġccc .': 1, 'ĠBbb Ġccc Ġddd': 2, 'Ġccc Ġddd .': 7, 'ĠBbb Ġccc Ġddd .': 3, 'ĠCCC Ġddd Ġeee': 4, 'Ġddd Ġeee .': 8, 'ĠCCC Ġddd Ġeee .': 5}


21

### Convert the type from numpy.float64 to numpy.float32

In [15]:
import numpy as np
from scipy.sparse import csr_matrix
tf_train = tf_train.astype(np.float32)

In [16]:
tf_test = tf_test.astype(np.float32)

### Convert the type from numpy.float64 to numpy.float16

In [15]:
# import numpy as np
# from scipy.sparse import csr_matrix

# ## convert data for tf_train
# data  = tf_train.data.astype(np.float16)
# indices = tf_train.indices
# indptr = tf_train.indptr
# tf_train = csr_matrix((data, indices, indptr), shape = tf_train.shape)


# ## convert data for tf_test
# data = tf_test.data.astype(np.float16)
# indices = tf_test.indices
# indptr = tf_test.indptr
# tf_test = csr_matrix((data, indices, indptr), shape = tf_test.shape)

# del data, indices, indptr
# gc.collect()

0

## Start training

In [16]:
### save train_vocab
import joblib
model_path = '/home/peng_sun2/s3shared/kaggle/llm-2023/baseline1/'
joblib.dump(train_vocab, f'{model_path}vocab_train.pkl')

['/home/peng_sun2/s3shared/kaggle/llm-2023/baseline1/vocab_train.pkl']

In [19]:
y_train = train['label'].values

In [20]:


# # Check if it's in scoring stage
# if len(test.text.values) <= 5:
#     # if not, just sample submission
#     sub.to_csv('submission.csv', index=False)
# else:
# otherwise, run fitting process
clf = MultinomialNB(alpha=0.02)
clf2 = MultinomialNB(alpha=0.01)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber") 
p6={
    'n_iter': 1500,
    'verbose': -1,
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05073909898961407, 
    'colsample_bytree': 0.726023996436955, 
    'colsample_bynode': 0.5803681307354022, 
    'lambda_l1': 8.562963348932286, 
    'lambda_l2': 4.893256185259296, 
    'min_data_in_leaf': 115, 
    'max_depth': 123, 
    'max_bin': 250,
    'device': 'gpu'
}
lgb=LGBMClassifier(**p6)

from catboost import CatBoostClassifier

cat=CatBoostClassifier(
    iterations=1000,
    verbose=0,
    l2_leaf_reg=6.6591278779517808,
    learning_rate=0.005689066836106983,
    allow_const_label=True
)
weights = [0.45, 0.45, 0.3, 0.45]
#weights = [0.1]
weights = [w/sum(weights) for w in weights]

# ensemble = VotingClassifier(
#     estimators=[
#       ('mnb',clf),
# #         ('sgd', sgd_model),
# #          ('lgb',lgb), 
# #         ('cat', cat)
#     ],
#     weights=weights, 
#     voting='soft', n_jobs=1)
    
    
    

In [19]:
# import joblib 
# ensemble.fit(tf_train, y_train )
# ## output the models
# model_path = '/home/peng_sun2/s3shared/kaggle/llm-2023/baseline1/'
# # joblib.dump(clf, f'{model_path}clf.pkl');
# # joblib.dump(sgd_model, f'{model_path}sgd_model.pkl')
# # joblib.dump(lgb, f'{model_path}lgb.pkl')
# # joblib.dump(cat, f'{model_path}cat.pkl')
# joblib.dump(ensemble, f'{model_path}ensemble.pkl');

In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import lightgbm
## cv split
skf = StratifiedKFold(n_splits = 5)
skf.get_n_splits(tf_train, y_train)

aus_list = []
for i, (train_idx, test_idx) in enumerate(skf.split(tf_train, y_train)):
    train_X = tf_train[train_idx]
    eval_X = tf_train[test_idx];
    train_y = y_train[train_idx]
    eval_y = y_train[test_idx];
    
    print(train_X.shape)
#     ensemble.fit(train_X, train_y)
#     joblib.dump(clf, f'{model_path}clf_cv_{i}.pkl');
#     joblib.dump(sgd_model, f'{model_path}sgd_model_cv_{i}.pkl')
#     joblib.dump(lgb, f'{model_path}lgb_cv_{i}.pkl')
#     joblib.dump(cat, f'{model_path}cat_cv_{i}.pkl')
#     joblib.dump(ensemble, f'{model_path}ensemble_cv_{i}.pkl');
#     eval_preds = ensemble.predict_proba(eval_X)[:,1]
    ## fit each model
    clf.fit(train_X, train_y);
    gc.collect()
    
    sgd_model.fit(train_X, train_y)
    gc.collect()
    
    lgb.fit(train_X, train_y)
    gc.collect()
    
    cat.fit(train_X, train_y)
    gc.collect()
    

    eval_preds = clf.predict_proba(eval_X)[:, 1]* weights[0] + sgd_model.predict_proba(eval_X)[:, 1]*weights[1] \
        + lgb.predict_proba(eval_X)[:, 1]*weights[2] + cat.predict_proba(eval_X)[:, 1]*weights[3]
    
    #eval_preds = ensemble.predict(eval_X)
    ## compute AUC
    fpr, tpr, thresholds = metrics.roc_curve(eval_y, eval_preds, pos_label = 1)
    print(fpr, tpr)
    print(metrics.auc(fpr, tpr))
    aus_list.append(metrics.auc(fpr, tpr))
    #break;
    

(35894, 20784)


/home/ubuntu/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296
[LightGBM] [Warning] num_iterations is set=1500, n_iter=1500 will be ignored. Current value: num_iterations=1500
[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.82648402e-04
 1.82648402e-04 3.65296804e-04 3.65296804e-04 5.47945205e-04
 5.47945205e-04 7.30593607e-04 7.30593607e-04 9.13242009e-04
 9.13242009e-04 1.09589041e-03 1.09589041e-03 1.27853881e-03
 1.27853881e-03 1.46118721e-03 1.46118721e-03 1.64383562e-03
 1.64383562e-03 1.82648402e-03 1.82648402e-03 2.00913242e-03
 2.00913242e-03 2.19178082e-03 2.19178082e-03 2.37442922e-03
 2.37442922e-03 2.55707763e-03 2.55707763e-03 2.73972603e-03
 2.73972603e

/home/ubuntu/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296
[LightGBM] [Warning] num_iterations is set=1500, n_iter=1500 will be ignored. Current value: num_iterations=1500
[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.82681768e-04
 1.82681768e-04 3.65363537e-04 3.65363537e-04 5.48045305e-04
 5.48045305e-04 7.30727073e-04 7.30727073e-04 9.13408842e-04
 9.13408842e-04 1.09609061e-03 1.09609061e-03 1.27877238e-03
 1.27877238e-03 1.46145415e-03 1.46145415e-03 1.64413592e-03
 1.64413592e-03 1.82681768e-03 1.82681768e-03 2.00949945e-03
 2.00949945e-03 2.19218122e-03 2.19218122e-03 2.37486299e-03
 2.37486299e-03 2.55754476e-03 2.55754476e-03 2.92290829e-03
 2.92290829e

...

[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296
[LightGBM] [Warning] num_iterations is set=1500, n_iter=1500 will be ignored. Current value: num_iterations=1500
[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.82681768e-04
 1.82681768e-04 3.65363537e-04 3.65363537e-04 5.48045305e-04
 5.48045305e-04 7.30727073e-04 7.30727073e-04 9.13408842e-04
 9.13408842e-04 1.09609061e-03 1.09609061e-03 1.27877238e-03
 1.27877238e-03 1.46145415e-03 1.46145415e-03 1.64413592e-03
 1.64413592e-03 1.82681768e-03 1.82681768e-03 2.00949945e-03
 2.00949945e-03 2.19218122e-03 2.19218122e-03 2.37486299e-03
 2.37486299e-03 2.55754476e-03 2.55754476e-03 2.74022653e-03
 2.74022653e

/home/ubuntu/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296
[LightGBM] [Warning] num_iterations is set=1500, n_iter=1500 will be ignored. Current value: num_iterations=1500
[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.82681768e-04
 1.82681768e-04 3.65363537e-04 3.65363537e-04 5.48045305e-04
 5.48045305e-04 7.30727073e-04 7.30727073e-04 9.13408842e-04
 9.13408842e-04 1.09609061e-03 1.09609061e-03 1.27877238e-03
 1.27877238e-03 1.46145415e-03 1.46145415e-03 1.64413592e-03
 1.64413592e-03 1.82681768e-03 1.82681768e-03 2.00949945e-03
 2.00949945e-03 2.19218122e-03 2.19218122e-03 2.37486299e-03
 2.37486299e-03 2.55754476e-03 2.55754476e-03 2.74022653e-03
 2.74022653e

/home/ubuntu/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296
[LightGBM] [Warning] num_iterations is set=1500, n_iter=1500 will be ignored. Current value: num_iterations=1500
[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[0.         0.         0.         ... 0.80471319 0.80471319 1.        ] [0.00000000e+00 2.85795942e-04 1.08888254e-01 ... 9.99714204e-01
 1.00000000e+00 1.00000000e+00]
0.9422863445613096


In [22]:
import sklearn 
import lightgbm
display(sklearn.__version__)
display(lightgbm.__version__)

'1.2.2'

'3.3.2'

In [22]:
# experiment with all models, without VotingEnsemble, self-coded soft voting, original parameter setting, with numpy.float32
aus_list

[0.9960866053053645,
 0.9947028028602746,
 0.9967226890756303,
 0.9973018544992708,
 0.9422863445613096]

In [21]:
# experiment with all models, without VotingEnsemble, self-coded soft voting, original parameter setting
aus_list

[0.9960500129847929,
 0.99474659428989,
 0.9968172660368496,
 0.9973122442311666,
 0.9431892070420871]

In [21]:
# experiment with all models, VotingEnsemble n_jobs = 1, original parameters setting, changed weights
aus_list

[0.9960656208362207,
 0.9947012370165457,
 0.9968239469700924,
 0.9973352687124032,
 0.9428879048171079]

In [20]:
# experiment with only multinomialNB, original parameter setting
aus_list

[0.9633644054857161,
 0.9755928284357223,
 0.9867114150007829,
 0.9750459523745131,
 0.7752189335791227]

In [20]:
# experiment with only catboost, original parameter setting
aus_list

[0.9951818197240959,
 0.988915444438645,
 0.9920609113210502,
 0.9955969986936087,
 0.9605266936228868]

In [21]:
# experiment with only lightgbm 4, with cpu max bin 900 and max_depth 123
aus_list

[0.9978818736207736,
 0.9945432433843102,
 0.9961483897906989,
 0.9974760260852231,
 0.9665304445771498]

[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296
[LightGBM] [Warning] num_iterations is set=1500, n_iter=1500 will be ignored. Current value: num_iterations=1500
[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115
[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286


In [20]:
# experiment with only lightgbm 3
aus_list

[0.9633644054857161,
 0.9755928284357223,
 0.9867114150007829,
 0.9750459523745131,
 0.7752189335791227]

In [21]:
#experiment with only lightgbm2, depth 23
aus_list

[0.9980322623163044,
 0.994754319118952,
 0.9964657341197348,
 0.9976234663006696,
 0.9670913856801093]

In [20]:
#experiment with only lightgbm, max_bin 250, max-depth = 123
aus_list

[0.9980014642148245,
 0.9947916383944883,
 0.9964219426901196,
 0.9976602219351152,
 0.9670607907912099]

In [22]:
#experiment wiht only lightgbm, max_bin 200
aus_list

[0.9979935819888527,
 0.9949521895714809,
 0.9962090923325854,
 0.997664973018545,
 0.966284693481503]

In [21]:
#experiment without catboost
aus_list

[0.9974415651699572,
 0.9950194686570281,
 0.996574873427632,
 0.9978453575597517,
 0.9644704583375404]

[LightGBM] [Warning] lambda_l1 is set=8.562963348932286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.562963348932286
[LightGBM] [Warning] lambda_l2 is set=4.893256185259296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.893256185259296
[LightGBM] [Warning] num_iterations is set=1500, n_iter=1500 will be ignored. Current value: num_iterations=1500
[LightGBM] [Warning] min_data_in_leaf is set=115, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=115


In [23]:
aus_list

[0.9976213425623237,
 0.9957595907928388,
 0.9970352314838978,
 0.9979617852086347,
 0.9608195900848752]

In [19]:
## with external data
aus_list

[0.9948483859465849,
 0.9940963507093242,
 0.9947070055108838,
 0.9951455995069962,
 0.9753977531773758]

In [19]:
## without external data
aus_list

[0.9974293503297094,
 0.9950473406754007,
 0.9966961219270317,
 0.9978349678278557,
 0.9645059087292857]